In [1]:
import requests 
import pandas as pd 
from datetime import date 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import os
import numpy as np

import pymongo
from pymongo import MongoClient
from IPython.display import clear_output 

from utils.misc import subcollection_to_df, read_mongo

%matplotlib inline

In [2]:
client = MongoClient()
db = client['prozorro']
collection='etenders'


df = read_mongo(db, collection='etenders')


In [3]:
# change dtypes
df.id = df.id.astype('string')
df.date = pd.to_datetime(df.date, utc=True)
df.dateModified = pd.to_datetime(df.dateModified, utc=True)
df.tenderID = df.tenderID.astype('string')
df.owner = df.owner.astype('category')
df.procurementMethod = df.procurementMethod.astype('category')
df.submissionMethod = df.submissionMethod.astype('category')
df.numberOfBids = df.numberOfBids.fillna(0).astype(int)
df.status = df.status.astype('category')
df.procurementMethodType = df.procurementMethodType.astype('category')
df.mainProcurementCategory = df.mainProcurementCategory.astype('category')

df=df[~df.value.isna()].reindex()

In [4]:
df['procuringEntity_name'] = df.procuringEntity.apply(lambda value: value['name']).astype("string")
df['procuringEntity_kind'] = df.procuringEntity.apply(lambda value: value['name']).astype("category")
df['procuringEntity_identifier'] = df.procuringEntity.apply(lambda value: value['identifier'])
df['procuringEntity_identifier_id'] = df['procuringEntity_identifier'].apply(lambda value: value['id']).astype("string")
df['procuringEntity_identifier_scheme'] = df['procuringEntity_identifier'].apply(lambda value: value['scheme']).astype("category")
df['procuringEntity_address'] = df.procuringEntity.apply(lambda value: value['address'])
df['procuringEntity_contactPoint'] = df.procuringEntity.apply(lambda value: value['contactPoint'])
df['procuringEntity_contactPoint_email'] = df['procuringEntity_contactPoint'].apply(lambda value: value.get('email',pd.NA)).astype("string")
df['procuringEntity_contactPoint_name'] = df['procuringEntity_contactPoint'].apply(lambda value: value['name']).astype("string")
df['procuringEntity_contactPoint_telephone'] = df['procuringEntity_contactPoint'].apply(lambda value: value.get('telephone',pd.NA)).astype("string")
df['procuringEntity_kind'] = df['procuringEntity'].apply(lambda value:  value.get('kind',pd.NA)).astype("category")
df['num_lots'] = df.lots.apply(lambda x: len(x) if isinstance(x, list) else 0)

## 

In [5]:
df['procuringEntity_address']

0        {'streetAddress': 'вул. Промислова, 133', 'loc...
1        {'streetAddress': 'вул. Студентська, буд. 2', ...
2        {'streetAddress': 'вул. Матросова 75', 'locali...
3        {'streetAddress': 'вул. М. Грушевського, 12/2,...
4        {'streetAddress': 'вул. С.Петлюри, 25, м. Київ...
                               ...                        
28994    {'streetAddress': 'вул. Ярослава Мудрого, 9', ...
28995    {'streetAddress': 'вул. Промислова, 9б', 'loca...
28996    {'streetAddress': '71701, м. Токмак, Запорізьк...
28997    {'streetAddress': 'вул. Матросова 75', 'locali...
28998    {'streetAddress': 'вул. Миколи Боровського, 28...
Name: procuringEntity_address, Length: 28967, dtype: object

In [7]:
#PE geolocation
from utils.geolocation import CachedGeolocator, region_fixer, locality_fixer, countryName_fixer, address_to_location
from functools import partial
fixers = {
    'locality': locality_fixer,
    'region': region_fixer,
    'countryName': countryName_fixer
}
geolocator = CachedGeolocator(cache_dir='./data/geo_cache', bad_cache_dir='./data/geo_cache_bad', verbose=False)

get_location = partial(address_to_location, geolocator=geolocator, fixers=fixers)
df['procuringEntity_geo'] = df.procuringEntity_address.apply(lambda address:get_location(address))
# tenderers_df['tenderer_point'] = tenderers_df.address.apply(lambda addr:get_location(addr))


geolocator.close()

In [16]:
bids_df = df[['id', 'bids']]
awards_df = df[['id', 'awards']]
items_df = df[['id', 'items']]

df.drop(columns=['procuringEntity_contactPoint', 'procuringEntity_identifier', 'procuringEntity', 'procuringEntity_address', 'bids', 'awards', 'items'], inplace=True)
df.dtypes

id                                                     string
date                                      datetime64[ns, UTC]
dateModified                              datetime64[ns, UTC]
tenderID                                               string
owner                                                category
procurementMethod                                    category
submissionMethod                                     category
numberOfBids                                            int64
status                                               category
procurementMethodType                                category
value                                                  object
lots                                                   object
mainProcurementCategory                              category
procuringEntity_name                                   string
procuringEntity_kind                                 category
procuringEntity_identifier_id                          string
procurin

In [17]:


# tenders with and without lots need to be processed defferently
df_w_lots = df[~df.lots.isna()]
df_wo_lots = df[df.lots.isna()]

### processing value

In [18]:
lots_df = subcollection_to_df(df, 'lots')
lots_df = lots_df[['id','status', 'value', 'tender_id']].rename(columns={'id':'lots_id', 'status':'lots_status', 'value':'lots_value'})
df_w_lots = df_w_lots.merge(lots_df, left_on="id", right_on="tender_id", suffixes=('','_'))

# value from lots
df_w_lots['value_amount'] = df_w_lots.lots_value.apply(lambda value: value['amount'])
df_w_lots['value_currency'] = df_w_lots.lots_value.apply(lambda value: value['currency']).astype('category')
df_w_lots['value_valueAddedTaxIncluded'] = df_w_lots.lots_value.apply(lambda value: value['valueAddedTaxIncluded']).astype(bool)

In [20]:
WITHOUT_LOTS = 'WITHOUT LOTS'
df_wo_lots['lots_id'] = WITHOUT_LOTS

# value from top-level
df_wo_lots['value_amount'] = df_wo_lots.value.apply(lambda value: value['amount'])
df_wo_lots['value_currency'] = df_wo_lots.value.apply(lambda value: value['currency']).astype('category')
df_wo_lots['value_valueAddedTaxIncluded'] = df_wo_lots.value.apply(lambda value: value['valueAddedTaxIncluded']).astype(bool)

/tmp/ipykernel_20048/703009612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_lots['lots_id'] = WITHOUT_LOTS
/tmp/ipykernel_20048/703009612.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_lots['value_amount'] = df_wo_lots.value.apply(lambda value: value['amount'])
/tmp/ipykernel_20048/703009612.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [21]:
df = pd.concat([df_w_lots, df_wo_lots])
df.drop(columns=['tender_id','lots_value', 'lots', 'value'], inplace=True)

del df_wo_lots
del df_w_lots

In [27]:
del lots_df

df['lots_id'] = df['lots_id'].astype('string')
df['lots_status'] = df['lots_status'].astype('category')
df['value_currency'] = df['value_currency'].astype('category')


### Bids

In [17]:

bids_df

,id,bids
0,54c44b64ee504474afaf1183c410b36c,"[{'tenderers': [{'name': 'ТОВ ФІРМА ""СОФТПРО""'..."
1,4edb5e732a6743d29382e62995cf9ad7,[{'tenderers': [{'name': 'Товариство з обмежен...
2,79d1b3f60ca64f6ea425c22ad0fc90f9,[{'tenderers': [{'name': 'Рекмістренко Валерій...
3,9be27fbba9614e89b38a508bafe61a7b,[{'tenderers': [{'name': 'Товариство з обмежен...
4,2acab13b39564fc4bc184d89e1225cbe,"[{'tenderers': [{'name': 'ТОВ ТВК Вектор-ВС', ..."
...,...,...
28994,52d8a15c55dd4f2ca9232f40c89bfa82,"[{'tenderers': [{'name': 'ТОВ СМКД', 'identifi..."
28995,2cf090528c064f439c6ed2e678636a27,[{'tenderers': [{'name': 'ФОП Бобровський Ігор...
28996,6a0585fcfb05471796bb2b6a1d379f9b,NaN
28997,d1c74ec8bb9143d5b49e7ef32202f51c,[{'tenderers': [{'name': 'Фізична особа-підпри...


In [29]:
bids_subdf = subcollection_to_df(bids_df, 'bids')
bids_subdf = bids_subdf[~bids_subdf.tenderers.isna()]

In [39]:
tenderers_subdf = subcollection_to_df(bids_subdf, 'tenderers')[[
'address',
'parent_id',
'contactPoint',
'identifier',
'name',
]]

In [41]:
tenderers_subdf['bids_tenderers_identifier_id'] = tenderers_subdf['identifier'].apply(lambda value: value['id']).astype("string")
tenderers_subdf['bids_tenderers_identifier_scheme'] = tenderers_subdf['identifier'].apply(lambda value: value['scheme']).astype("category")

tenderers_subdf['bids_tenderers_contactPoint_email'] = tenderers_subdf['contactPoint'].apply(lambda value: value.get('email',pd.NA)).astype("string")
tenderers_subdf['bids_tenderers_contactPoint_name'] = tenderers_subdf['contactPoint'].apply(lambda value: value['name']).astype("string")
tenderers_subdf['bids_tenderers_contactPoint_telephone'] = tenderers_subdf['contactPoint'].apply(lambda value: value.get('telephone',pd.NA)).astype("string")

,address,parent_id,bids_tenderers_name,bids_tenderers_identifier_id,bids_tenderers_identifier_scheme,bids_tenderers_contactPoint_email,bids_tenderers_contactPoint_name,bids_tenderers_contactPoint_telephone
0,"{'streetAddress': 'проспект Гагаріна, б. 13', ...",575755ff2ab14506860d6cc5665628ae,"ТОВ ФІРМА ""СОФТПРО""",14112174,UA-EDR,altiy@wgsoftpro.com,Землицький Альтій Юхимович,+380577602925
1,"{'streetAddress': 'вул,Жовтнева,66', 'locality...",185913a23d834953b6521fc9e0d10de9,ТОВ ІНФОСВІТ ІТ СЕРВІС,37061252,UA-EDR,bond@infoservis.pl.ua,Бондаренко Олег Михайлович,+380503040299
2,"{'streetAddress': 'вул. Ломоносова, 26', 'loca...",e038b7528c7644b9880f6cf1f2d2551b,"Товариство з обмеженою відповідальністю ""Торго...",32358958,UA-EDR,tdmsopm@kmk.kharkov.ua,Ганжа Роман Віталійович,+38 (050) 402-1259
3,"{'streetAddress': '61129, Харківська обл., міс...",9f9741c316b4419697b0b5b78cb01390,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ФУДСНАБ""",38385713,UA-EDR,ev.mironenko11@gmail.com,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ФУДСНАБ""",0960345308
4,"{'streetAddress': 'ВУЛИЦЯ БУЧМИ, будинок 32Б, ...",3bde3a9d385946cebbd99f9e7441769b,ФОП Юркевич Антон Ігорович,3159106257,UA-EDR,antonur@meta.ua,Юркевич Антон,"380508030906, 380508030906"
...,...,...,...,...,...,...,...,...
42796,"{'streetAddress': 'с. Суховоля', 'locality': '...",e696cbb55928462c932d39d88a1ff218,ФОП Лепецький Володимир Ігорович,2600304558,UA-EDR,klasikinterier@gmail.com,Лепецький Володимир Ігорович,+380674230908
42797,"{'streetAddress': 'ж/м Тополя-3, буд.20, корпу...",18859547e7a54d7f979e418dcec9bab6,Фізична особа-підприємець Воробйова Ольга Мико...,2810900040,UA-EDR,FOPVorobyovaOlga@ukr.net,Воробйова Ольга Миколаївна,"380669821034, 380669821034"
42798,"{'streetAddress': 'вул. Зоотехнічна, 2', 'loca...",9bbc1bbf354646bda2eb862acf07bca2,ТОВ ФУД ПОСТАЧ,41505858,UA-EDR,41505858@ukr.net,Ільїн Владислав Олександрович,+380675588677
42799,"{'streetAddress': '65031, Одеська обл., місто ...",6a03cd140e2e42d3a361255a5cc8058b,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""СТАР ...",40806547,UA-EDR,star-mital@ukr.net,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""СТАР ...",+380681205182


In [42]:
geolocator = CachedGeolocator(cache_dir='./data/geo_cache', bad_cache_dir='./data/geo_cache_bad', verbose=False)
# get_location = partial(address_to_location, geolocator=geolocator, fixers=fixers)

tenderers_subdf['bids_tenderers_geo'] = tenderers_subdf.address.apply(lambda addr:get_location(addr))
geolocator.close()

In [56]:
tenderers_subdf = tenderers_subdf.rename(columns={'name':'bids_tenderers_name'}).drop(columns=['identifier','contactPoint', 'address'])

In [58]:
tenderers_subdf.shape

(42801, 8)

In [60]:
bids_subdf = bids_subdf.merge(tenderers_subdf, left_on='id', right_on='parent_id')

In [66]:
bids_subdf = bids_subdf.rename(columns = {
    'date':'bids_date',
    'status':'bids_status'
})[[
'bids_date',
'bids_id',
'value',
'lotValues',
'bids_status',
'tender_id',
'bids_tenderers_name',                       
'bids_tenderers_identifier_id',               
'bids_tenderers_identifier_scheme',        
'bids_tenderers_contactPoint_email',       
'bids_tenderers_contactPoint_name',        
'bids_tenderers_contactPoint_telephone',
'bids_tenderers_geo',
]]

In [70]:
bids_subdf = bids_subdf.rename(columns = {
    'bids_id':'id'
})

In [71]:
# separate bids subcollection
bids_w_lots_df = bids_subdf[~bids_subdf.lotValues.isna()]
bids_wo_lots_df = bids_subdf[bids_subdf.lotValues.isna()]

# del bids_subdf
# del bids_df

In [72]:
# nested bid values from lots
bids_lotValues_df = subcollection_to_df(bids_w_lots_df, 'lotValues')
bids_lotValues_df['bids_value_amount'] = bids_lotValues_df.value.apply(lambda value: value['amount'])
bids_lotValues_df['bids_value_currency'] = bids_lotValues_df.value.apply(lambda value: value['currency']).astype('category')
bids_lotValues_df['bids_value_valueAddedTaxIncluded'] = bids_lotValues_df.value.apply(lambda value: value['valueAddedTaxIncluded']).astype(bool)


bids_lotValues_df = bids_lotValues_df[['relatedLot', 'parent_id', 'bids_value_amount', 'bids_value_currency', 'bids_value_valueAddedTaxIncluded']]

bids_w_lots_df = bids_w_lots_df.merge(bids_lotValues_df, left_on="id", right_on="parent_id", suffixes=('','_'))

del bids_lotValues_df


In [75]:
bids_wo_lots_df = bids_wo_lots_df[~bids_wo_lots_df.value.isna()]

bids_wo_lots_df['bids_value_amount'] = bids_wo_lots_df.value.apply(lambda value: value['amount'])
bids_wo_lots_df['bids_value_currency'] = bids_wo_lots_df.value.apply(lambda value: value['currency']).astype('category')
bids_wo_lots_df['bids_value_valueAddedTaxIncluded'] = bids_wo_lots_df.value.apply(lambda value: value['valueAddedTaxIncluded']).astype(bool)

bids_wo_lots_df['relatedLot'] = WITHOUT_LOTS

In [86]:
ddddd_df = pd.concat([bids_w_lots_df, bids_wo_lots_df])

In [87]:
ddddd_df = ddddd_df.drop(columns=['value', 'lotValues']).rename(columns={'relatedLot':'lots_id', 'id':'bids_id'})

In [90]:
df_merged = df.merge(ddddd_df, left_on=('id', 'lots_id'), right_on=('tender_id', 'lots_id'), suffixes=('','_bids'), how='left').reindex()

df_merged = df_merged.drop(columns=['parent_id', 'tender_id'])

In [93]:
df_merged.dtypes

id                                                     object
date                                      datetime64[ns, UTC]
dateModified                              datetime64[ns, UTC]
tenderID                                               string
owner                                                category
procurementMethod                                    category
submissionMethod                                     category
numberOfBids                                            int64
status                                               category
procurementMethodType                                category
mainProcurementCategory                              category
procuringEntity_name                                   string
procuringEntity_kind                                 category
procuringEntity_identifier_id                          string
procuringEntity_identifier_scheme                    category
procuringEntity_contactPoint_email                     string
procurin

### Awards

In [100]:
awards_subdf = subcollection_to_df(awards_df, 'awards')

awards_subdf = awards_subdf.rename(columns={
    'id':'awards_id',
    'date':'awards_date',
    'status':'awards_status',
    'lotID': 'lots_id',
    'bid_id':'bids_id'
})[[
    'awards_id', 'awards_date', 'awards_status', 'bids_id', 'tender_id', 'lots_id', 'value'
]]

awards_subdf['awards_value_amount'] = awards_subdf.value.apply(lambda value: value['amount'])
awards_subdf['awards_value_currency'] = awards_subdf.value.apply(lambda value: value['currency']).astype('category')
awards_subdf['awards_value_valueAddedTaxIncluded'] = awards_subdf.value.apply(lambda value: value['valueAddedTaxIncluded']).astype(bool)


awards_subdf = awards_subdf.drop(columns=['value'])

awards_subdf.lots_id.fillna(WITHOUT_LOTS, inplace=True)

In [ ]:
del awards_subdf
del awards_df

In [102]:
df_merged = df_merged.merge(awards_subdf, left_on=('id', 'lots_id', 'bids_id'), right_on=('tender_id', 'lots_id', 'bids_id'), suffixes=('','_awards'), how='left').reindex()

In [106]:
df_merged = df_merged.drop(columns=['tender_id'])

### Items

In [107]:
items_subdf = subcollection_to_df(items_df, 'items')

In [109]:
items_subdf['code'] = items_subdf.classification.apply(lambda x: x.get('id',''))
items_subdf.relatedLot.fillna(WITHOUT_LOTS, inplace=True)

items_subdf = items_subdf.rename(columns={
    'relatedLot': 'lots_id'
})

items_subdf = items_subdf[[
    'tender_id', 'lots_id', 'code'
]]


In [110]:
# first from items, in most cases items came from the same category
items_subdf = items_subdf.groupby(['tender_id','lots_id']).agg(lambda x: list(x)[0]).reset_index()

In [111]:
from utils.dk021 import DK021

classifier = DK021.load('./data/dk021.csv')

items_subdf['items_classification_l0'] = items_subdf.code.apply(lambda code: classifier.get_level_category(code=code, level=1).code)
items_subdf['items_classification_l1'] = items_subdf.code.apply(lambda code: classifier.get_level_category(code=code, level=2).code)
items_subdf['items_classification_l2'] = items_subdf.code.apply(lambda code: classifier.get_level_category(code=code, level=3).code)

In [114]:
df_merged = df_merged.merge(items_subdf, left_on=('id', 'lots_id'), right_on=('tender_id', 'lots_id'), suffixes=('','_itemss'), how='left').reindex()

In [116]:
df_merged = df_merged.drop(columns=['tender_id'])

In [122]:
df_merged.dtypes

id                                                     object
date                                      datetime64[ns, UTC]
dateModified                              datetime64[ns, UTC]
tenderID                                               string
owner                                                category
procurementMethod                                    category
submissionMethod                                     category
numberOfBids                                            int64
status                                               category
procurementMethodType                                category
mainProcurementCategory                              category
procuringEntity_name                                   string
procuringEntity_kind                                 category
procuringEntity_identifier_id                          string
procuringEntity_identifier_scheme                    category
procuringEntity_contactPoint_email                     string
procurin

In [124]:
df_merged.bids_date = pd.to_datetime(df_merged.bids_date, utc=True)
df_merged.id = df_merged.id.astype('string')
df_merged.bids_status = df_merged.bids_status.astype('category')
df_merged.bids_tenderers_name = df_merged.bids_tenderers_name.astype('category')
df_merged.bids_value_currency = df_merged.bids_value_currency.astype('category')
df_merged.bids_value_valueAddedTaxIncluded = df_merged.bids_value_valueAddedTaxIncluded.astype(bool)
df_merged.awards_id = df_merged.awards_id.astype('string')
df_merged.awards_date = pd.to_datetime(df_merged.awards_date, utc=True)
df_merged.awards_value_valueAddedTaxIncluded = df_merged.awards_value_valueAddedTaxIncluded.astype(bool)
df_merged.awards_id = df_merged.awards_id.astype('string')
df_merged.code = df_merged.code.astype('category')
df_merged.items_classification_l0 = df_merged.items_classification_l0.astype('category')
df_merged.items_classification_l1 = df_merged.items_classification_l1.astype('category')
df_merged.items_classification_l2 = df_merged.items_classification_l2.astype('category')


In [127]:
df_merged.to_parquet('df.parquet.gzip', compression='gzip') 

In [129]:
df_restored.shape

(56877, 48)

In [128]:
df_restored = pd.read_parquet('df.parquet.gzip')



id                                                     string
date                                      datetime64[ns, UTC]
dateModified                              datetime64[ns, UTC]
tenderID                                               string
owner                                                category
procurementMethod                                    category
submissionMethod                                     category
numberOfBids                                            int64
status                                               category
procurementMethodType                                category
mainProcurementCategory                              category
procuringEntity_name                                   string
procuringEntity_kind                                 category
procuringEntity_identifier_id                          string
procuringEntity_identifier_scheme                    category
procuringEntity_contactPoint_email                     string
procurin

In [119]:
df_merged.dtypes

id                                                     object
date                                      datetime64[ns, UTC]
dateModified                              datetime64[ns, UTC]
tenderID                                               string
owner                                                category
procurementMethod                                    category
submissionMethod                                     category
numberOfBids                                            int64
status                                               category
procurementMethodType                                category
mainProcurementCategory                              category
procuringEntity_name                                   string
procuringEntity_kind                                 category
procuringEntity_identifier_id                          string
procuringEntity_identifier_scheme                    category
procuringEntity_contactPoint_email                     string
procurin

In [118]:
from geopy import distance
df_merged['geo_distance']=df_merged.apply(lambda x: distance.distance(x.procuringEntity_geo, x.bids_tenderers_geo).km , axis=1)
# df_merged['geo_distance'].plot.hist();

ValueError: A single number has been passed to the Point constructor. This is probably a mistake, because constructing a Point with just a latitude seems senseless. If this is exactly what was meant, then pass the zero longitude explicitly to get rid of this error.